# Aula 9

## *Bounded model checking* (BMC) para *First-order Transition Systems* (FOTS)

FOTS são modelos de sistemas dinâmicos que são determinados por um espaço de estados, uma relação de transição de estados e um conjunto de estados iniciais. Nos FOTS o conjunto de estados iniciais é descrito por um predicado unário (*init*) sobre o vector de variáveis associado ao espaço de estados, e a relação de transição é descrita por um predicado binário (*trans*) sobre dois vectores de variáveis que representam o estado antes e depois da transição.

### Modelação de programas com FOTS

Um programa pode ser modelado por um FOTS da seguinte forma:
- O estado é constituído pelas variáveis do programa mais uma variável para o respectivo *program counter*
- Os estados iniciais são caracterizados implicitamente por um predicado sobre as variáveis de estado
- As transições são caracterizadas implicitamente por um predicado sobre pares de estados

Considere, por exemplo o programa seguinte, anotado com uma pré-condição que restringe o valor inicial de $x$:

```Python
{x >= 3}
0: while (x>0):
1:    x = x-1
2:
```

Neste caso o estado do FOTS respectivo será um par de inteiros, o primeiro contendo o valor do $\mathit{pc}$ (o *program counter* que neste caso pode ser 0, 1 ou 2) e o segundo o valor da variável $x$. O estado inicial é caracterizado pelo seguinte predicado:

$$
\mathit{pc} = 0 \wedge x \ge 3
$$

As transições possíveis no FOTS são caracterizadas pelo seguinte predicado:

$$
\begin{array}{c}
(\mathit{pc} = 0 \wedge x > 0 \wedge \mathit{pc}' = 1 \wedge x' = x)\\
\vee\\
(\mathit{pc} = 0 \wedge x \le 0 \wedge \mathit{pc}' = 2 \wedge x' = x) \\
\vee\\
(\mathit{pc} = 1 \wedge \mathit{pc}' = 0 \wedge x' = x - 1) \\
\vee\\
(\mathit{pc} = 2 \wedge \mathit{pc}' = 2 \wedge x' = x)
\end{array}
$$

Note que este predicado é uma disjunção de todas as possíveis transições que podem ocorrer no programa. Cada transição é caracterizada por um predicado onde uma variável do programa denota o seu valor no pré-estado e a mesma variável com apóstrofe denota o seu valor no pós-estado. É usual exigir que cada estado do FOTS tenha pelo menos um sucessor, pelo que o estado final do programa é caracterizado por uma transição para ele próprio (um lacete).

Usando estes predicados podemos usar um SMT solver (nomeadamente o Z3) para, por exemplo, gerar uma possível execução de $k\!-\!1$ passos do programa (em que $k>0$). Para tal precisamos de criar $k$ cópias das variáveis que caracterizam o estado do FOTS e depois impor que a primeira cópia satisfaz o predicado inicial e que cada par de cópias consecutivas satisfazem o predicado de transição.

A seguinte função cria a $i$-ésima cópia das variáveis de estado, agrupadas num dicionário que nos permite aceder às mesmas pelo nome.

In [1]:
from z3 import *

def declare(i):
    state = {}
    state['pc'] = Int('pc'+str(i))
    state['x'] = Int('x'+str(i))
    return state

### Exercício 1

Defina a função `init` que, dado um possível estado do programa (um dicionário de variáveis), devolva um predicado Z3 que testa se esse estado é um possível estado inicial do programa.

In [2]:
def init(s):
    return And(s['pc']==0,s['x']>=3)

### Exercício 2

Defina a função `trans` que, dados dois possíveis estados do programa, devolva um predicado Z3 que testa se é possível transitar do primeiro para o segundo.

In [3]:
def trans(s,p):
    t01= And(s['pc']==0,s['x']>0,p['pc']==1,p['x']==s['x'])
    t02= And(s['pc']==0,s['x']<=0,p['pc']==2,p['x']==s['x'])
    t10= And(s['pc']==1,p['pc']==0,p['x']==s['x']-1)
    t22= And(s['pc']==2,p['pc']==2,p['x']==s['x'])
    return Or(t01,t02,t10,t22)

### Exercício 3

Complete a função de ordem superior `gera_traco` que, dada uma função que gera uma cópia das variáveis do estado, um predicado que testa se um estado é inicial, um predicado que testa se um par de estados é uma transição válida, e um número positivo `k`, use o Z3 para gerar um possível traço de execução do programa de tamanho `k`. Para cada estado do traço deverá imprimir o respectivo valor das variáveis.

In [4]:
def gera_traco(declare,init,trans,k):
    s=Solver()
    state=[]
    for i in range(k):
        state.append(declare(i))
    s.add(init(state[0]))
    for i in range(k-1):
        s.add(trans(state[i],state[i+1]))
    if s.check()==sat:
        m=s.model()
        for i in range(k):
            print(i)
            for v in state[i]:
                print(v,m[state[i][v]])
                
gera_traco(declare,init,trans,10)

0
pc 0
x 5
1
pc 1
x 5
2
pc 0
x 4
3
pc 1
x 4
4
pc 0
x 3
5
pc 1
x 3
6
pc 0
x 2
7
pc 1
x 2
8
pc 0
x 1
9
pc 1
x 1


### Lógica temporal linear (LTL)

Sobre este FOTS podemos querer verificar várias propriedades temporais, como por exemplo:
1. $x$ é sempre positivo
2. $x$ é sempre diferente de 1
3. $x$ chega inevitavelmente a 0
4. $x$ chega inevitavelmente a um valor negativo
5. o programa termina

Estas propriedades são de natureza muito diferente. As duas primeiras são *propriedades de segurança* que, em geral, garantem que "nada de mau irá acontecer". Neste caso em particular são invariantes, ou seja, propriedades que devem ser verdade em todos os estados da execução do programa. As três últimas são *propriedades de animação* que, em geral, garantem que "algo de bom irá acontecer".

A lógica LTL introduz *operadores temporais* que nos permitem escrever estas propriedades formalmente. Os operadores mais conhecidos são o $G$, que informalmente significa "*é sempre verdade que*", e o $F$, que informalmente significa "*é inevitável que*". Com estes operadores, as propriedades acima podem ser especificadas formalmente do seguinte modo
1. $G\ (x \ge 0)$
2. $G\ (x \neq 1)$
3. $F\ (x = 0)$
4. $F\ (x < 0)$
5. $F\ (pc = 2)$

### BMC para LTL

Como é óbvio, nem todas estas propriedades são válidas. Em particular a 2ª e a 4ª não o são. O objectivo da verificação é precisamente determinar se uma propriedade temporal é válida num FOTS ou não. Este procedimento designa-se *model checking* e, quando uma propriedade não é válida, produz um contra-exemplo (um traço do FOTS correspondente a uma execução do programa onde a propriedade falha). Nesta aula vamos estudar uma técnica particular de *model checking* designada *bounded model checking*, onde o objectivo é determinar se uma propriedade temporal é válida nos primeiros $K$ estados da execução do FOTS.

A ideia passa por usar um SMT solver (neste caso o Z3) para tentar descobrir um contra-exemplo para uma dada propriedade. Para simplificar vamos abordar apenas dois casos: verificação de invariantes da forma $G\ \phi$ e propriedades de animação simples da forma $F\ \phi$, em que $\phi$ é uma fórmula sem operadores temporais.

### BMC de invariantes

Para fazer BMC de um invariante, por exemplo $G\ (x \ge 0)$ basta-nos usar o Z3 para encontrar um contra-exemplo com no máximo $K$ estados onde a propriedade $(x \ge 0)$ seja inválida nalgum estado. Para tal podemos implementar um procedimento iterativo que tenta encontrar esse contra-exemplo com tamanhos crescentes, começando com tamanho 1 até ao tamanho máximo $K$. Para cada tamanho $0 < k \le K$ basta tentar encontrar um possível traço onde o invariante a verificar seja inválido no último estado. Para tal podemos usar um código muito semelhante ao da função `gera_traco`. O procedimento é interrompido mal um contra-exemplo seja encontrado, sendo garantido que esse é um contra-exemplo mínimo para essa propriedade.

### Exercício 4

Complete a definição da função de ordem superior `bmc_always` que, dada uma função que gera uma cópia das variáveis do estado, um predicado que testa se um estado é inicial, um predicado que testa se um par de estados é uma transição válida, um invariante a verificar, e um número positivo `K`, use o Z3 para verificar se esse invariante é sempre válido nos primeiros `K-1` passos de execução do programa, ou devolva um contra-exemplo mínimo caso não seja.

In [5]:
def bmc_always(declare,init,trans,inv,K):
    for k in range(1,K+1):
        print("k=",k)
        s=Solver()
        state=[]
        for i in range(k):
            state.append(declare(i))
        s.add(init(state[0]))
        for i in range(k-1):
            s.add(trans(state[i],state[i+1]))
        s.add(Not(inv(state[k-1])))
        if s.check()==sat:
            m=s.model()
            for i in range(k):
                print(i)
                for v in state[i]:
                    print(v,m[state[i][v]])
    print ("O invariante e sempre valido nos primeiros",k,"estados")
def positive(state):
    return (state['x'] >= 0)

bmc_always(declare,init,trans,positive,20)

k= 1
k= 2
k= 3
k= 4
k= 5
k= 6
k= 7
k= 8
k= 9
k= 10
k= 11
k= 12
k= 13
k= 14
k= 15
k= 16
k= 17
k= 18
k= 19
k= 20
O invariante e sempre valido nos primeiros 20 estados


### Exercício 5

Use o procedimento `bmc_always` para encontrar um contra-exemplo para a segunda propriedade especificada acima.

In [6]:
# completar

### Exercício 6

Considere o seguinte programa (em pseudo-código) concorrente com 2 processos que utilizam um semáforo (uma variável global booleana partilhada entre os dois) para controlar o acesso exclusivo a um recurso crítico que só pode ser acedido por um deles de cada vez (por exemplo, um ficheiro onde pretendem ambos escrever). Ambos os processos tentam repetidamente executar esse código crítico alternadamente com código não crítico (ambos abstraídos por um simples comentário neste programa). O procedimento de controlo de acesso à região crítica é aqui abstraído pela execução atómica de uma espera activa até que o semáforo seja `True` seguida de uma atribuição ao mesmo do valor `False`.

```Python
{ while True:
0:    while (not sem); sem = False 
1:    # critical code of process 1
2:    sem = True
3:    # non critical code of process 2
}
||
{ while True:
0:    while (not sem); sem = False
1:    # critical code of process 2
2:    sem = True
3:    # non critical code of process 2
}
```

Mostre como pode utilizar o procedimento `bmc_always` para verificar a propriedade fundamental de segurança deste código: os dois processos nunca podem estar simultaneamente na região crítica. Note que, como temos dois processos, é conveniente usar dois *program counters* no estado, um por cada processo.

In [ ]:
# completar

### BMC de propriedades de animação

Para fazer BMC de propriedades de animação da forma $F\ \phi$ o prodimento é ligeiramente mais complicado. Neste caso não basta encontrar uma traço aberto com num máximo $K$ estados onde $\phi$ nunca seja válida, pois tal contra-exemplo não seria convincente: nada garante que $\phi$ não pudesse ser válida num ponto mais tarde da execução. Neste caso, o contra-exemplo teria que ser uma execução completa do programa que demonstre inequivocamente que a propriedade não é válida. É possível encontrar uma execução completa do programa com no máximo $K$ estados se a mesma tiver um *loop* no último estado, mais concretamente, uma transição para um dos estados precedentes. 

### Exercício 7

Complete a definição da função de ordem superior `bmc_eventually` que, dada uma função que gera uma cópia das variáveis do estado, um predicado que testa se um estado é inicial, um predicado que testa se um par de estados é uma transição válida, uma propriedade cuja inevitabilidade se pretende verificar, e um número positivo `K`, use o Z3 para encontrar um contra-exemplo para essa propriedade considerando apenas os primeiros `K` estados de execução do programa. Note que neste caso um contra-exemplo tem que ser necessariamente um *loop* (no sentido acima referido) onde a propriedade desejada nunca seja válida.

In [ ]:
def bmc_eventually(declare,init,trans,prop,bound):
    # completar

def zero(state):
    return (state['x'] == 0)

bmc_eventually(declare,init,trans,zero,20)

def terminates(state):
    return (state['pc'] == 2)

bmc_eventually(declare,init,trans,terminates,20)

### Exercício 8

Modifique a função `bmc_eventually` para não só imprimir os estados do contra-exemplo, mas também o estado onde começa o *loop*. Sugere-se a utilização da função `eval` do Z3 para detectar esse estado. Utilize esta função modificada para encontrar um contra exemplo para a quarta propriedade acima referida.

In [ ]:
# completar